In [161]:
import pandas as pd
import numpy as np
import pymongo
import requests
import json
import pickle
from collections import defaultdict
from collections import Counter
from scipy.special import logit, expit

In [5]:
def reccomend(a1=None,a2=None,a3=None,a4=None,a5=None,
              e1=None,e2=None,e3=None,e4=None,e5=None,
              enemy_laner1=None,enemy_laner2=None):
    
    allies=[champid for champid in [a1,a2,a3,a4,a5] if champid != None]
    enemies=[champid for champid in [e1,e2,e3,e4,e5] if champid != None]
    return allies,enemies

In [6]:
result=reccomend(a1=1,a2=8,e1=133,e2=178,e3=203)
result

([1, 8], [133, 178, 203])

In [8]:
try:
    champ_dict
except:
    champ_dict = pickle.load(open("champion_dict.p","rb"))

def champion_number_to_name(champion_number):

    return champ_dict["data"][str(champion_number)]["name"]

In [9]:
possible_champs=champ_dict["data"].keys()
predicting_probs={}

In [390]:
class recommender(object):
    def __init__(self,a1=None,a2=None,a3=None,a4=None,a5=None,
                 e1=None,e2=None,e3=None,e4=None,e5=None,
                 enemy_laner1=None,enemy_laner2=None):
        #self.a1=a1, self.a2=a2, self.a3=a3, self.a4=a4, self.a5=a5
        #self.e1=e1, self.e2=e2, self.e3=e3, self.e4=e4, self.e5=e5
        self.enemy_laners=[champid for champid in [enemy_laner1,enemy_laner2] if champid != None]
        self.allies=[champid for champid in [a1,a2,a3,a4,a5] if champid != None]
        self.enemies=[champid for champid in [e1,e2,e3,e4,e5] if champid != None]
        self.champion_dict = pickle.load(open("champion_dict.p","rb"))
        self.matchup_win_percent= pickle.load(open("matchup_win_percent.p","rb"))
        self.synergy_win_percent= pickle.load(open("synergy_win_percent.p","rb"))
        self.flat_win_percent = pickle.load( open( "flat_win_percent.p", "rb" ) )
        self.lane_matchup= pickle.load( open( "  ", "rb" ) )
        self.predicted_winrates = {}
        self.update_events=0
        
        
    def load_base_winrate(self):
        for key in self.flat_win_percent:
            self.predicted_winrates[key]=logit(.5) #self.flat_win_percent[key]

###START
    def get_winrate_for_enemy(self,possible_pick,enemyid):
        return self.matchup_win_percent[possible_pick,enemyid]
    def get_winrate_for_ally(self,possible_pick,allyid):
        return self.synergy_win_percent[possible_pick,allyid]
    def get_winrate_for_enemy_laner(self, possible_pick,lanerid):
        return self.lane_matchup[possible_pick,lanerid]
    
    
    def update_predicted_winrate_for_enemy(self,picked_champion):
        self.update_events+=1   
        for champion in self.predicted_winrates.keys():
            if champion == picked_champion:
                pass
            else:
                self.predicted_winrates[champion] = self.predicted_winrates[champion] + logit(self.get_winrate_for_enemy(champion,picked_champion))    
    
    def update_predicted_winrate_for_ally(self,picked_champion):
        self.update_events+=1
        for champion in self.predicted_winrates.keys():
            if champion == picked_champion:
                pass
            else:
                self.predicted_winrates[champion] = self.predicted_winrates[champion] + logit(self.get_winrate_for_ally(champion,picked_champion))    
    
    def update_predicted_winrate_for_lane(self, enemy_champion):
        self.update_events+=1
        for champion in self.predicted_winrates.keys():
            self.predicted_winrates[champion] = self.predicted_winrates[champion] + logit(self.get_winrate_for_enemy_laner(champion,picked_champion))
###END           
    
    def top_picks(self):
        return sorted(self.predicted_winrates.items(), key=lambda(item): item[1], reverse=True)
    
    def champion_name_from_id(self,champion_number):
        return self.champion_dict["data"][str(champion_number)]["name"]
    
    def predict(self,n=5):
        top_picks=[]
        for pair in self.top_picks()[:n]:
            top_picks.append((self.champion_name_from_id(pair[0]), expit(pair[1]/(self.update_events+1))))
        return top_picks
        
        
        

In [394]:
model=recommender(a1=1,a2=8,e1=3,e2=4,e3=26)

In [395]:
model.predicted_winrates

{}

In [399]:
sorted(model.synergy_win_percent.items(),reverse=True, key=lambda x: x[1])

[((20, 421), 1.0),
 ((72, 60), 1.0),
 ((103, 103), 1.0),
 ((203, 72), 1.0),
 ((120, 72), 1.0),
 ((72, 203), 1.0),
 ((421, 20), 1.0),
 ((136, 42), 1.0),
 ((72, 120), 1.0),
 ((127, 112), 1.0),
 ((72, 33), 1.0),
 ((112, 127), 1.0),
 ((39, 39), 1.0),
 ((77, 60), 1.0),
 ((48, 48), 1.0),
 ((134, 134), 1.0),
 ((33, 72), 1.0),
 ((34, 34), 1.0),
 ((127, 30), 1.0),
 ((36, 36), 1.0),
 ((60, 77), 1.0),
 ((119, 119), 1.0),
 ((18, 18), 1.0),
 ((69, 69), 1.0),
 ((60, 72), 1.0),
 ((21, 21), 1.0),
 ((30, 127), 1.0),
 ((78, 78), 1.0),
 ((42, 136), 1.0),
 ((203, 32), 0.9),
 ((32, 203), 0.9),
 ((136, 103), 0.9),
 ((103, 136), 0.9),
 ((240, 122), 0.875),
 ((203, 154), 0.875),
 ((60, 120), 0.875),
 ((122, 240), 0.875),
 ((120, 60), 0.875),
 ((154, 203), 0.875),
 ((127, 163), 0.8571428571428571),
 ((163, 127), 0.8571428571428571),
 ((33, 203), 0.8181818181818182),
 ((203, 33), 0.8181818181818182),
 ((20, 56), 0.8),
 ((40, 201), 0.8),
 ((164, 58), 0.8),
 ((58, 164), 0.8),
 ((201, 40), 0.8),
 ((154, 120), 0.8)

In [350]:
model.load_base_winrate()

In [351]:
model.predict()

Annie 0.5
Olaf 0.5
Galio 0.5
Twisted Fate 0.5
Xin Zhao 0.5


In [352]:
model.enemies, model.allies

([3, 4, 26], [1, 8])

In [353]:
model.update_predicted_winrate(8,enemy=False)

In [354]:
model.predict()

Tahm Kench 0.561523675802
Evelynn 0.547957842623
Viktor 0.54446657822
Kennen 0.54196010845
Jax 0.541872729702


In [355]:
model.update_predicted_winrate(40)

In [356]:
model.predict()

Tahm Kench 0.568406123971
Evelynn 0.558579471314
Viktor 0.554606431244
Zilean 0.551232306671
Camille 0.55093926085


In [357]:
model.update_predicted_winrate(9)

In [358]:
model.predict()

Ryze 0.571786991032
Cassiopeia 0.559481860222
Corki 0.558665552711
Karthus 0.557682453201
Nunu 0.55669039197


In [359]:
model.update_predicted_winrate(57,enemy=False)

In [360]:
model.predict()

Ryze 0.572244656349
Tahm Kench 0.562200837852
Nidalee 0.555119791733
Cassiopeia 0.553612221434
Poppy 0.551810774787


In [320]:
projected = sorted(flat_wr.items(), key=lambda x: x[1], reverse=True)
for pair in projected[:5]:
    print champion_name_from_id(pair[0]),pair[1]

Ryze 0.576932111816
Nunu 0.559818625741
Evelynn 0.546513856415
LeBlanc 0.545976120329
Nidalee 0.545306001188


In [392]:
def run_and_predict(a1=None,a2=None,a3=None,a4=None,a5=None,
                 e1=None,e2=None,e3=None,e4=None,e5=None,
                 enemy_laner1=None,enemy_laner2=None):
    model=recommender(**locals())
    model.load_base_winrate()
    for champ in model.enemies:
        model.update_predicted_winrate_for_enemy(champ)
    for champ in model.allies:
        model.update_predicted_winrate_for_ally(champ)
    for champ in model.enemy_laners:
        model.update_predicted_winrate_for_enemy_laner(champ)
    for line in model.predict():
        print line

In [393]:
run_and_predict(a1=1,a2=8,e1=3,e2=4,e3=26,a3=39)

(u'LeBlanc', 0.54681470124788178)
(u'Cassiopeia', 0.54569302404224018)
(u'Nunu', 0.54250770535922155)
(u'Taliyah', 0.5419628873453548)
(u'Urgot', 0.54042647222995377)
